In [ ]:
import pyspark.sql as ps
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
import pandas
import matplotlib.pyplot as plt
inp_file_path: str = 'file:///D:/MAHE/5thSem/BDA2/data/2019-Oct.csv'
print(inp_file_path)

file:///D:/MAHE/5thSem/BDA2/data/2019-Oct.csv


In [69]:
spark: ps.SparkSession = ps.SparkSession.builder.master("yarn").appName("test").getOrCreate()
raw_data: ps.DataFrame = spark.read.csv(inp_file_path, header=True, inferSchema=True)

In [70]:
raw_data.show(5)
raw_data.summary()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 05:30:00|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 05:30:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 05:30:01|      view|  17200506|2053013559792632471|furniture.living_...|    null|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 05:30:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 05:30:04|      view|   1004237|2053013555631882655|electr

DataFrame[summary: string, event_type: string, product_id: string, category_id: string, category_code: string, brand: string, price: string, user_id: string, user_session: string]

In [ ]:
str_ind = StringIndexer(inputCols=['event_type', 'category_code'], outputCols=['event_ind', 'category_ind'], handleInvalid='keep')
str_ind = str_ind.fit(raw_data)
eve_dict = {label: ind for ind, label in enumerate(str_ind.labelsArray[0])}
cat_dict = {label: ind for ind, label in enumerate(str_ind.labelsArray[1])}

dataset = str_ind.transform(raw_data)
dataset = dataset.drop(dataset.category_id, dataset.event_type, dataset.category_code). \
    withColumnsRenamed({'event_ind': 'event_type', 'category_ind': 'category_code'}). \
    withColumn('event_time', F.to_date(dataset.event_time))

In [ ]:
print(eve_dict)
print(cat_dict)
dataset.show()
print(dataset.summary())

{'view': 0, 'cart': 1, 'purchase': 2}
{'electronics.smartphone': 0, 'electronics.clocks': 1, 'computers.notebook': 2, 'electronics.video.tv': 3, 'electronics.audio.headphone': 4, 'appliances.kitchen.refrigerators': 5, 'appliances.kitchen.washer': 6, 'appliances.environment.vacuum': 7, 'apparel.shoes': 8, 'auto.accessories.player': 9, 'computers.desktop': 10, 'apparel.shoes.keds': 11, 'furniture.bedroom.bed': 12, 'electronics.tablet': 13, 'electronics.audio.subwoofer': 14, 'furniture.living_room.cabinet': 15, 'construction.tools.drill': 16, 'electronics.telephone': 17, 'auto.accessories.videoregister': 18, 'kids.carriage': 19, 'furniture.living_room.sofa': 20, 'appliances.kitchen.blender': 21, 'appliances.kitchen.oven': 22, 'auto.accessories.alarm': 23, 'accessories.bag': 24, 'appliances.kitchen.kettle': 25, 'appliances.kitchen.microwave': 26, 'appliances.iron': 27, 'appliances.kitchen.meat_grinder': 28, 'appliances.environment.air_heater': 29, 'appliances.sewing_machine': 30, 'computer

DataFrame[summary: string, product_id: string, brand: string, price: string, user_id: string, user_session: string, event_type: string, category_code: string]

In [ ]:
visits = dataset.groupBy(F.to_date(dataset.event_time).alias("event_time")). \
    agg(F.count_distinct("user_session").alias("Number_of_daily_visits"),
        F.count_distinct("user_id").alias("Number_of_daily_visitors"))

sales = dataset.where(dataset.event_type == eve_dict['purchase']). \
    withColumn("event_time", F.to_date(dataset.event_time)). \
    groupBy("event_time"). \
    agg(F.count("event_type").alias("number_of_daily_sales"),
        F.sum("price").alias("total_daily_sales"))

daily = visits.join(sales, on=["event_time"], how="left"). \
    withColumn("conversion_rate", visits.number_of_daily_sales / visits.number_of_daily_visits)

daily.show()

In [ ]:
print(daily.describe('number_of_daily_visits'))

plt.plot(daily.select(daily.event_time).toPandas(),
         daily.select(daily.number_of_daily_visits).toPandas())
plt.title('Daily Visits')
plt.ylabel('Number of Daily Visits')
plt.xlabel('Dates')
plt.show()

plt.boxplot(daily.withColumn('day', F.date_format(daily.event_time, 'E')).select(daily.day).toPandas(),
            daily.select(daily.number_of_daily_visits).toPandas())
plt.title('Number of Visits by Days')
plt.ylabel('Number of Visits')
plt.xlabel('Days')
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.show()

In [ ]:
print(daily.describe('number_of_daily_visitors'))

plt.plot(daily.select(daily.event_time).toPandas(),
         daily.select(daily.number_of_daily_visitors).toPandas())
plt.title('Daily Visitors')
plt.ylabel('Number of Daily Visitors')
plt.xlabel('Dates')
plt.show()

plt.boxplot(daily.withColumn('day', F.date_format(daily.event_time, 'E')).select(daily.day).toPandas(),
            daily.select(daily.number_of_daily_visitors).toPandas())
plt.title('Number of Visitors by Days')
plt.ylabel('Number of Visitors')
plt.xlabel('Days')
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.show()

In [ ]:
print(daily.describe('conversion_rate'))

plt.plot(daily.select(daily.event_time).toPandas(),
         daily.select(daily.conversion_rate).toPandas())
plt.title('Daily Conversion Rate')
plt.ylabel('Conversion Rate')
plt.xlabel('Dates')
plt.show()

plt.boxplot(daily.withColumn('day', F.date_format(daily.event_time, 'E')).select(daily.day).toPandas(),
            daily.select(daily.conversion_rate).toPandas())
plt.title('Conversion Rate by Days')
plt.ylabel('Conversion Rate')
plt.xlabel('Days')
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.show()

In [ ]:
customers = dataset.where(dataset.event_type == eve_dict['purchase']). \
    groupBy('user_id'). \
    agg(F.count('user_id').alias('number_of_purchases'),
        F.sum('price').alias('total_sales'))
customers.show()

In [ ]:
customers_who_purchased = customers.select(customers.user_id). \
    dropDuplicates(). \
    count()

repeat_customers = customers.where('number_of_purchases > 1'). \
    select(customers.user_id). \
    dropDuplicates(). \
    count()
    
print(f'There are {customers_who_purchased} customers, who purchased in October, out of these {repeat_customers} are repeat customers.')

print('Distribution of Customer by Number of Purchases')
customers.groupBy(customers.number_of_purchases).count().show(10)
print(customers.describe())

In [ ]:
sales_thresholds = customers.approxQuantile("total_sales", [0.01, 0.95], 0.0)
customers_filtered = customers.filter(customers.total_sales >= sales_thresholds[0] &
                                      customers.total_sales <= sales_thresholds[1])

plt.hist(customers_filtered.select(customers_filtered.total_sales).toPandas(), 50)

In [ ]:
top10p_sales = customers.approxQuantile("total_sales", [0.9], 0.0)[0]
top10p_customers = customers.filter(F.col("total_sales") >= top10p_sales)
regular_customers = customers.filter(F.col("total_sales") < top10p_sales)

print(r'Top 10% customers Purchase Amount-Descriptive Statistics')
print(top10p_customers.select(top10p_customers.total_sales).describe())

In [ ]:
spark.stop()